# UMR MBT Microbiome Praktikum Session 2

---

## _Aims_

- How do the samples group with each other? Principal Components
- How much diversity is there in a sample?  Alpha diversity
- How does the diversity differ between samples? Beta diversity
- What is the taxonomic content of the sample?  With reference only to known taxa; Including unknown taxa
- Rarefaction: Adjusting for differences in library sizes across samples to aid comparisons

Reference: Willis A. 2019. _Front. Microbiol._ 10: 2407, https://doi.org/10.3389/fmicb.2019.02407

---

## _Why R for Bioinformatics?_

- Freely available
- User- friendly resources to learn R
- R editors and integrated development environments (IDEs)
- Active user community
- Large code repositories: CRAN, Bioconductor, github, gitlab...
- Ready-made packages and functions for Bioinformatics
- Reproducibility: Code, library (versions), Rmarkdown, Jupyter notebooks


---

## 2.1 Loading libraries and microbiome data #####

First, we start our R environment by loading the needed packages. For this, we need to install first Bioconductor and then the [`phyloseq`](http://joey711.github.io/phyloseq/) package (ca. 5-15 minutes).

In [ ]:
install.packages("BiocManager")
BiocManager::install("phyloseq")

Now we load our required libraries:

In [ ]:
# load ggplot2 library (graphics)
library(ggplot2)

# loading phyloseq library (microbiome analysis)
library(phyloseq)

Then we load the needed input files: the OTU table ([BIOM format](https://biom-format.org/)) and the metadata (tab- separated)

In [ ]:
# OTU data
InputBiomFile <- "https://raw.githubusercontent.com/barrantesisrael/mbtmicrobiome2023/refs/heads/main/data2025/mbtmicrobiome2024.biom"

# Samples' metadata
InputMapFile <- "https://raw.githubusercontent.com/barrantesisrael/mbtmicrobiome2023/refs/heads/main/data2025/sample-metadata-2024.tsv"

# prepare phyloseq object by loading both files
BiomData <- import_biom(InputBiomFile, parseFunction = parse_taxonomy_greengenes)
SampleData <- import_qiime_sample_data(InputMapFile)

Our metadata contains the categories and groups that distinguish the samples:

In [ ]:
head(SampleData)

Now we proceed to build a `phyloseq` object by combining both the OTU table and the metadata:

In [ ]:
# create phyloseq object by merging OTU and sample data
ExperimentPhyloseqObject <- merge_phyloseq(BiomData, SampleData)

We call our object to see its properties:

In [ ]:
# checking the features of our original microbiome data
ExperimentPhyloseqObject

Now we create another phyloseq object, that includes only those taxa with more than 100 amplicons in all samples, and with `Gender` in the metadata

In [ ]:
# create a temporary phyloseq object for working
psTemp <- ExperimentPhyloseqObject

# subset samples
# Prune OTUs with low abundances from all samples
psTemp <- prune_taxa(taxa_sums(psTemp) > 100, psTemp)

# Prune samples with no metadata
psTemp <-  subset_samples(psTemp, Gender != "U")

# checking the features of our microbiome data
psTemp

_Q: What are the differences between the `ExperimentPhyloseqObject` and `psTemp` objects?_

---

## 2.2 Sample ordination

- Ordination is a visualization technique that places samples in 2D or 3D space based on their microbial community similarity. Samples with similar microbiomes appear close together; different ones are far apart.
- Ordination shows quickly if samples cluster by treatment, disease state, or other factors—revealing biological patterns in your data at a glance.
- **PCoA (Principal Coordinates Analysis)**: Works with ecological distances like Bray-Curtis dissimilarity, which better captures compositional differences between communities

**References:**
- Bray & Curtis (1957). An ordination of the upland forest communities. *Ecological Monographs*, doi:10.2307/1942268
- Ramette (2007). Multivariate analyses in microbial ecology. *FEMS Microbiology Ecology*, doi:10.1111/j.1574-6941.2007.00375.x

---

_Hands on_: Using distance methods we will see how similar (or dissimilar) are the samples in the context of their microbiomes. The output generated by `distance()` will be afterwards transformed into a two-dimensional data through the `ordination()`function.

In [ ]:
# Calculate distance and ordination
iDist <- distance(psTemp, method="bray")
iMDS  <- ordinate(psTemp, distance=iDist)

# plot sample ordination, e.g. by Gender
plot_ordination(psTemp, iMDS, color="Gender")

_Q: Are there any clear separations between the gender groups? Why/Why not?_

Repeat the ordination using other metadata, e.g. "Cat":

In [ ]:
plot_ordination(psTemp, iMDS, color="Cat") +
  geom_text(aes(label=SampleID), vjust = -1)

---

## 2.3 Microbial communities

### _Phylum and genus barplots_

- Microbial communities are usually represented by bar plots showing the relative abundance (percentage) of different bacterial groups (phyla or genera) in each sample. Each color represents a different taxon, and bar height shows how much of the community it comprises.
- After taxonomy assignment, sequences are grouped by taxonomic level (phylum, genus, etc.) and counted. The counts are converted to percentages so all bars add up to 100%, allowing direct comparison between samples regardless of sequencing depth.
- Provides an intuitive visual summary of community composition—you can immediately see which bacteria dominate each sample and how communities differ. For example, a healthy gut might show high *Bacteroidetes*, while a diseased one shows more *Proteobacteria*.
- Typically only the top 10-20 most abundant taxa are shown individually; rare taxa are grouped as "Other" to avoid cluttered plots.

**References:**
- Caporaso et al. (2010). QIIME allows analysis of high-throughput community sequencing data. *Nature Methods*, doi:10.1038/nmeth.f.303
- McMurdie & Holmes (2013). phyloseq: An R package for reproducible interactive analysis. *PLOS ONE*, doi:10.1371/journal.pone.0061217

---

### _Rarefaction_

- Normalization technique that randomly subsamples all samples to the same sequencing depth (number of reads) to enable fair comparison. Without this, samples with more reads appear artificially more diverse.
- Different samples often have different total read counts due to technical variation in sequencing. A sample with 50,000 reads will naturally detect more species than one with 10,000 reads—even if they're biologically identical.
- All samples are randomly downsampled to match the sample with the fewest reads. If your smallest sample has 10,000 reads, all other samples are rarefied to exactly 10,000 reads by random selection without replacement.
- **However:** Some argue rarefaction discards valuable data and recommend alternative normalization methods (e.g., CSS, DESeq2), particularly for differential abundance testing (McMurdie & Holmes, 2014, doi:10.1371/journal.pcbi.1003531)

**References:**
- Hughes et al. (2001). Counting the uncountable. *Applied and Environmental Microbiology*, doi:10.1128/AEM.67.10.4399-4406.2001
- McMurdie & Holmes (2014). Waste not, want not. *PLOS Computational Biology*, doi:10.1371/journal.pcbi.1003531

---

_Hands on_: Now first let's observe the absolute abundance on each sample at the Phylum level.

In [ ]:
### Abundances per sample
plot_bar(psTemp, "SampleID", fill="Phylum")

This plot above shows that the samples are not directly comparable. Hence we will perform a [rarefaction](https://www.frontiersin.org/journals/microbiology/articles/10.3389/fmicb.2019.02407/full), a method used to adjust the differences in library sizes, and then plot again:

In [ ]:
# initializes the random number generator to a specific starting point to ensure reproducibility
set.seed(1212)

# Rarefaction to an even depth
ps.rarefied <- rarefy_even_depth(psTemp)

# Agglomerate the data to a single taxonomic level, e.g. Phylum level
ps.rarefied.glom <- tax_glom(ps.rarefied, "Phylum")

# Plot abundances
plot_bar(ps.rarefied.glom, "SampleID", fill="Phylum")

Q: _What are the differences between the abundance plots before and after rarefaction?_

We can also plot the relative abundances according to any of the metadata categories, e.g. by `Gender`:

In [ ]:
### Abundances per category, e.g. "Gender"
# Merge samples by the selected category
mergedGP <- merge_samples(psTemp, "Gender")

# Rarefaction to an even depth
ps.rarefied <- rarefy_even_depth(mergedGP)

# Remove lines
ps.rarefied.glom <- tax_glom(ps.rarefied, "Phylum")

# Plot abundances for the example category "Gender"
plot_bar(ps.rarefied.glom, fill="Phylum")

---

## 2.4 Diversity

- **Diversity**: How many different species are present and how evenly distributed they are in microbial communities.
- **Alpha diversity**: Diversity **within** a single sample, e.g. "How diverse is this one community?"
  - Common metrics include species richness (total number of species), Shannon index (accounts for evenness), and observed ASVs/OTUs (Magurran, 2004, doi:10.1002/9780470999738).
  - The Shannon index combines richness and evenness: communities where all species have similar abundances score higher than those dominated by just a few species. Higher values = more diverse communities.

- **Beta diversity**: Diversity **between** samples, e.g. "How different are these communities from each other?"
  - Measured using distance metrics like Bray-Curtis dissimilarity or UniFrac distances (Lozupone & Knight, 2005, doi:10.1128/AEM.71.12.8228-8235.2005). Used to create ordination plots (PCoA, PCA).
  - Alpha = diversity within one sample (a single number per sample). Beta = compositional dissimilarity between pairs of samples (visualized as distance matrices or ordination plots).
  - Low alpha diversity may indicate dysbiosis; high beta diversity between groups suggests different environmental conditions or disease states are shaping distinct communities.

**References:**

- Magurran (2004). *Measuring Biological Diversity*. Blackwell Publishing, doi:10.1002/9780470999738
- Lozupone & Knight (2005). UniFrac: a new phylogenetic method. *Applied and Environmental Microbiology*, doi:10.1128/AEM.71.12.8228-8235.2005

---

_Hands on_: First let's observe the Shannon diversity on the individual samples:

In [ ]:
plot_richness(psTemp, x = "SampleID", measures = c("Shannon"))

 Repeat the same analyses at the Gender level:

In [ ]:
plot_richness(psTemp, x = "Gender", color = "Gender", measures = c("Shannon"))

A more appropriate view of our plot can be achieved by adding more features, e.g.:

In [ ]:
# Assign our plot to a variable
Our_Richness_plot <- plot_richness(psTemp, x = "Gender", color = "Gender", measures = c("Shannon"))

# Improving our plot e.g. by adding proper labels
Our_Richness_plot + geom_boxplot(data = Our_Richness_plot$data, aes(x = Gender, y = value, color = Gender), alpha = 0.1) + # boxplot
  labs(title = "Richness of the gut microbiome (Shannon alpha diversity)", subtitle = "MBT Class 2024") + # title and subtitle
  theme(axis.text.x = element_text(angle = 0, hjust = 0.5)) # x-axis labels: 0 degree rotation, 0.5 horizontal position

---

## Contact

Dr. rer. nat. Israel Barrantes <br>
Research Group Translational Bioinformatics (head)<br>
Institute for Biostatistics and Informatics in Medicine and Ageing Research, Office 3017<br>
Rostock University Medical Center<br>
Ernst-Heydemann-Str. 8<br>
18057 Rostock, Germany<br>

Email: israel.barrantes[bei]uni-rostock.de

---
Last update 2025/10/13